# 분류 이론
저번 세션에서 배운 분류 이론을 복습해봅시다.

노션에 있는 분류 이론 파트를 참고해도 좋아요! 

내용을 다시 읽어보면서 정리한다는 느낌으로 문제를 풀어주세요.

##
***머신러닝이 무엇인지 설명하세요.***

답: 모델에 데이터를 넣어주면 학습 후 일정 패턴을 찾아 예측해주는 과정

##
***머신러닝에는 지도학습과 비지도학습이 있습니다. 지도학습과 비지도학습의 차이를 설명해주세요.***

답: 지도학습은 y레이블에 정답값을 주고 학습을 진행하지만 비지도학습은 정답값이 따로 없고 데이터의 고유 특성을 분석해 분류한다.

##
***대표적인 지도학습 모델로는 회귀와 분류가 있습니다. 회귀와 분류의 차이를 설명해주세요.***

답: 회귀는 연속형 데이터를 주로 사용하여 회귀선을 작성해 예측하는것이 목표이고, 분류는 범주형 데이터를 주로 사용해 데이터를 분류하는것이 목적이다.

##
***이진분류와 다중분류의 차이를 설명해주세요.***

답: 이진분류는 답이 참/거짓 두가지로 나뉠때 사용되고, 다중분류는 답이 3가지 이상일 때 사용된다.

##
***세션에서 공부한 네 종류의 분류 모델을 간략히 설명해주세요.***

답 :
로지스틱은 회귀선에 로지스틱 함수를 추가해줘 0,1로 분류하는것이다. 시그모이드 함수를 추가할수도 있다.
의사결정나무는 조건에 따라 데이터를 분류하고 최종적으로 지니지수가 떨어질때까지 진행한다.
SVM의 경우 데이터를 분리하는 최적의 초평면을 찾는 모델이며, 데이터간 거리가 멀수록 좋은 초평면이다.
KNN은 기준 데이터로부터 거리를 정하고 그 거리안에 다른 데이터들의 숫자를 센다. 이후 가장많은 수의 데이터들과 같은 특성을 가졌다고 예측한다. 

# 분류 실습: 탑승한 항구를 예측하는 다중 분류 모델 만들기
저번 세션 시간에, 타이타닉 데이터셋을 이용해 **생존 여부(Survived)** 를 예측하는 분류 모델을 만들었었습니다.

그 모델은 **Survived/Not Survived** 를 예측하는 **이진 분류 모델** 이었습니다.

이번 과제에서는, **탑승한 항구(Embarked)** 를 예측하는 **다중 분류 모델** 을 만들어 볼 것입니다.

(탑승한 항구 컬럼의 값은 S, C, Q로 나뉘기 때문에, '이진 분류'가 아닌 '다중 분류'를 사용합니다.)

📌 어떤 사람에 대한 정보가 주어졌을 때, **그 사람이 어떤 항구에서 탑승했는지 예측하는 모델**만들어봅시다.
- 주어지는 정보: 생존 여부, 좌석 등급, 성별, 나이 등의 정보 (모델의 독립 변수)
- 예측하고자 하는 정보: 탑승한 항구 (모델의 종속 변수)

[ 변수 설명 ]

- PassengerId : 각 승객의 고유 번호

- Survived : 생존 여부(종속 변수)

        0 = 사망
        1 = 생존
 
- Pclass : 객실 등급 - 승객의 사회적, 경제적 지위

        1st = Upper
        2nd = Middle
        3rd = Lower

- Name : 이름

- Sex : 성별

- Age : 나이

- SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수

- Parch : 동반한 Parent(부모) Child(자식)의 수

- Ticket : 티켓의 고유넘버

- Fare : 티켓의 요금

- Cabin : 객실 번호

- Embarked : 승선한 항

## 데이터 읽기 및 전처리

In [1]:
# seaborn을 sns, pandas를 pd, numpy를 np로 import해주세요
import seaborn as sns
import pandas as pd
import numpy as np

# 타이타닉 데이터셋을 불러와서 df에 저장해주세요
df = pd.read_csv("./titanic1.csv")

In [2]:
# initial 컬럼을 만들고 일시적으로 값을 0으로 초기화
df['Initial'] = 0

for index, row in df.iterrows():
    initial_search = row['Name'].split(',')[1].split('.')[0].strip() # Name 컬럼에서 .(dot)을 기준으로 알파벳 문자열 추출
    df.at[index, 'Initial'] = initial_search

C:\Users\김숙경\AppData\Local\Temp\ipykernel_7776\2791606022.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Mr' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Initial'] = initial_search


In [3]:
# 유추 가능한 값들로 대체하고, 흔하지 않은 Initial들은 Other로 대체하겠습니다.
df['Initial'].replace([
    'Mlle', 'Mme', 'Ms', 'Dr', 'Major', 'Lady', 'Countess', 'Jonkheer', 'Col',
    'Rev', 'Capt', 'Sir', 'Don','the Countess' 
], [
    'Miss', 'Miss', 'Miss', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Other', 'Other',
    'Other', 'Mr', 'Mr', 'Mr', 'Other'
],
    inplace=True)

C:\Users\김숙경\AppData\Local\Temp\ipykernel_7776\2395359540.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Initial'].replace([


In [4]:
# 결측값을 Initial별 평균값으로 대체
df.loc[(df['Age'].isnull()) & (df.Initial == 'Mr'), 'Age'] = 33
df.loc[(df['Age'].isnull()) & (df.Initial == 'Mrs'), 'Age'] = 36
df.loc[(df['Age'].isnull()) & (df.Initial == 'Master'), 'Age'] = 5
df.loc[(df['Age'].isnull()) & (df.Initial == 'Miss'), 'Age'] = 22
df.loc[(df['Age'].isnull()) & (df.Initial == 'Other'), 'Age'] = 46

In [5]:
# Embarked 열의 결측값을 제거해주세요
df.dropna(subset=['Embarked'],axis=0,how='any',inplace=True)

# 'Cabin', 'Name', 'PassengerId', 'Ticket' 열은 분석에서 제외하겠습니다.
df.drop(['Cabin','Name','PassengerId','Ticket'],axis=1,inplace=True)

In [6]:
df['Embarked'].isnull().sum()

np.int64(0)

In [7]:
# 나중에 사용하기 위해 원본 데이터프레임을 저장해둡니다.
df_org = df.copy()

# SibSp 행과 Parch 행을 이용해 Relatives 열을 만듭니다.
df['Relatives'] = df["SibSp"] + df["Parch"]

In [8]:
# Sex 열 인코딩
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
# Age 열을 10년 단위로 나누어 인코딩
df['Age'] = (df['Age'] // 10).astype(int)
# Fare 열을 9분위로 구간화하고 인코딩
df['Fare'] = pd.qcut(df['Fare'], q=9, labels=range(9))
# Embarked 열 인코딩
df['Embarked'] = df['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})
# Initial 열 인코딩
initial_mapping = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Other':4}
df['Initial'] = df['Initial'].map(initial_mapping)

## 다중 분류 모델 target과 feature 정의

앞서 말했지만, 우리의 목표는 어떤 사람의 정보가 주어졌을 때 그 사람이 탑승한 항구를 예측하는 것입니다. 모델의 target과 feature가 무엇인지 정의해주세요.

- feature(예측을 위해 주어지는 정보 = 독립 변수): ***(답) 한 사람에 대한 정보 = Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Initial, Relatives***
- target(예측하고자 하는 값 = 종속 변수): ***(답) Embarked***

이 target과 feature에 따라 생존 여부를 예측하는 함수 predict_survival를 정의합니다.

이 함수에 model과 독립 변수들을 넣어주면, 승선한 항 예측 값과 확률을 반환합니다.

- 함수의 input: model, scaler, 독립 변수(pclass, sex, age, sibsp, parch, fare, survived, initial)
- 함수의 output: 승선한 항구, 확률

In [9]:
# predict_survival 함수 정의
def predict_survival(model, scaler, survived, pclass, sex, age, sibsp, parch, fare, initial):
    # 입력된 데이터를 데이터프레임으로 변환합니다.
    # 위에서 피처엔지니어링한 방식대로(모델의 input으로 적절하게) input 값을 변환해줍니다. 
    input_data = pd.DataFrame({
        'Survived': [survived],
        'Pclass': [pclass],
        'Sex': [0 if sex == 'male' else 1],  # 성별을 인코딩합니다.
        'Age': [age // 10],  # 나이를 10년 단위로 나눕니다.
        'SibSp': [sibsp],
        'Parch': [parch],
        'Fare': [fare],  # 요금을 일단 그대로 둡니다.
        'Initial': [0 if initial == 'Mr' else (1 if initial == 'Miss' else (2 if initial == 'Mrs' else (3 if initial == 'Master' else 4)))],  # 호칭을 인코딩합니다.
        'Relatives': [sibsp + parch],
    })
    
    # 'Fare' 값을 qcut으로 생성된 bins를 사용해 범주화합니다.
    fare_bins = pd.qcut(df_org['Fare'], 9, retbins=True)[1]  # pd.qcut을 사용해 요금 구간을 얻습니다.
    input_data['Fare'] = pd.cut(input_data['Fare'], bins=fare_bins, labels=False, include_lowest=True)


    # 입력 데이터를 스케일링합니다.
    input_data_scaled = scaler.transform(input_data)
    
    # 예측을 수행합니다.
    prediction = model.predict(input_data_scaled)
    prediction_proba = model.predict_proba(input_data_scaled)

    # 예측 결과를 반환합니다.
    result = "S" if prediction == 1 else ("C" if prediction == 2 else "Q")
    probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.
    
    return result, probability

## target과 feature 분리
데이터 셋을 target과 feature를 분리합니다.
예측하고자 하는 값인 target과 예측하기 위해 주어진 값인 feature를 각각 변수에 담습니다.

- target = 종속 변수 (변수명=y): 'Embarked'
- feature = 독립 변수 (변수명=X): 'Pclass', 'Sex', 'Age', 'Sibsp', 'Parch', 'Fare', `Survived', 'Initial'

[ 참고 ]

drop() 함수는 drop한 후의 데이터프레임을 반환합니다.
drop('drop하고자 하는 열', axis=1)

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [10]:
# 변수 X에 feature(= 'Embarked' 열을 drop한 데이터프레임)를 담아주세요.
X=df.drop('Embarked',axis=1)

In [11]:
X

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Initial,Relatives
0,0,3,0,2,1,0,0,0,1
1,1,1,1,3,1,0,7,2,1
2,1,3,1,2,0,0,2,1,0
3,1,1,1,3,1,0,7,2,1
4,0,3,0,3,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...
886,0,2,0,2,0,0,3,4,0
887,1,1,1,1,0,0,6,1,0
888,0,3,1,2,1,2,5,1,3
889,1,1,0,2,0,0,6,0,0


In [12]:
# 변수 y에 target(= 'Embarked' 열)을 담아주세요.
y = df['Embarked']

In [13]:
y

0      1
1      2
2      1
3      1
4      1
      ..
886    1
887    1
888    1
889    2
890    3
Name: Embarked, Length: 889, dtype: int64

## 데이터 셋을 훈련 세트와 테스트 세트로 나누기
- 훈련 세트 -> 모델을 학습시키는 데 사용됩니다.
- 테스트 세트 -> 완성된 모델을 평가하는 데 사용됩니다.

독립변수(X)와 종속변수(y)를 모두 훈련 세트와 테스트 세트로 나누어봅시다.

[ 변수명 ]

- 독립변수 훈련 세트: X_train
- 독립변수 테스트 세트: X_test
- 종속변수 훈련 세트: y_train
- 종속변수 테스트 세트: y_test

[ 참고 ]

tran_test_split 함수는 변수를 훈련 세트와 테스트 세트로 나누어 반환합니다.
~~~
train_test_split(독립변수, 종속변수, test_size=테스트 세트 크기, random_state=시드값)
~~~
-> 독립변수 훈련 세트, 독립변수 테스트 세트, 종속변수 훈련 세트, 종속변수 테스트 세트 반환

- test_size: 테스트 세트의 크기를 결정합니다. 예를 들어, 훈련 세트와 테스트 세트를 각각 70%와 30%로 하고 싶으면, test_size=0.3으로 하면 됩니다.
- random_state: 임의의 숫자로 설정된 시드로, 어떤 숫자를 사용해도 상관없습니다.

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [14]:
# sklearn.model_selectio의 train_test_split 함수를 import 해주세요.
from sklearn.model_selection import train_test_split

# tran_test_split 함수를 이용하여 독립변수(X)와 종속변수(y)를 각각 훈련 세트와 테스트 세트로 나누어주세요. 각 훈련 세트와 테스트 세트의 변수 명은 아래와 같습니다.
# 독립변수 훈련 세트: X_train, 독립변수 테스트 세트: X_test, 종속변수 훈련 세트: y_train, 종속변수 테스트 세트: y_test
# 단, random_state는 42로 합니다.
X_train,X_test,y_train,y_test = train_test_split( X, y, test_size=0.2, random_state=42)

# feature에 scaler 적용
MinMaxScaler를 이용해서 모든 feature(독립변수, X)를 스케일링 해줍니다. 

[ 변수명 ]
- 스케일링된 독립변수 훈련 세트: X_train_scaled
- 스케일링된 독립변수 테스트 세트: X_test_scaled

[ 참고 ]

1. MinMaxScaler 함수는 MinMaxScaler를 반환합니다.
~~~
MinMaxScaler()
~~~
2. fit_transform 함수는 훈련 데이터의 스케일링을 수행하고, 스케일링된 데이터를 반환합니다.
~~~
스케일러.fit_transform(훈련 데이터)
~~~
3. transform 함수는 테스트 데이터의 스케일링을 수행하고, 스케일링된 데이터를 반환합니다.
~~~
스케일러.transform(테스트 데이터)
~~~

cf) 훈련 데이터에 대해서는 fit과 transform을, 테스트 데이터에 대해서는 transform만을 수행합니다.

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [15]:
# sklearn.preprocessing에서 MinMaxScaler를 import 해옵니다.
from sklearn.preprocessing import MinMaxScaler

# scaler라는 변수를 MinMaxScaler 함수로 선언합니다.
scaler = MinMaxScaler()

# X_train_scaled라는 변수에 스케일링된 독립변수 훈련 세트를 저장합니다.
X_train_scaled= scaler.fit_transform(X_train)

# X_test_scaled라는 변수에 스케일링된 독립변수 테스트 세트를 저장합니다.
X_test_scaled= scaler.transform(X_test)

## 모델 생성 및 평가
위에서 만든 함수에 input으로 들어갈 model을 만듭니다. 총 4개의 다중 분류 모델을 만들어볼 것입니다.
1. 로지스틱 회귀
2. 의사 결정 나무
3. 서포트벡터머신(SVM)
4. kNN

또한, 생성한 모델들을 다음과 같은 방식으로 평가합니다.
1. accuracy
2. 분류 보고서

cf) 혼동 행렬은 이진분류에 대해 만들어지므로, 이번 실습에서 사용하지 않습니다.

### 로지스틱 회귀
로지스틱 회귀 이진 분류 모델을 생성합니다.
sklearn 라이브러리는 로지스틱 회귀 모델(LogisticRegression)을 제공합니다.

[ 참고 ]
로지스틱 회귀 모델 생성 함수
~~~
LogisticRegression()

- penalty: 어떤 방식의 규제를 적용할지 선택합니다.

- C: 규제의 강도를 조절하는 파라미터입니다.

- solver: 모델의 최적 가중치를 찾기 위해 사용하는 최적화 알고리즘을 선택합니다.
~~~
위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [16]:
# sklearn.linear_model에서 LogisticRegression 함수를 import 해오세요.

from sklearn.linear_model import LogisticRegression
# lr_model을 변수명으로 해서 로지스틱회귀 모델 객체를 생성하세요.
lr_model =  LogisticRegression()


이제 모델을 학습하고, 평가해보도록 하겠습니다.

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 모델 학습
lr_model.fit(X_train_scaled, y_train)

# 예측 결과 생성
lr_pred = lr_model.predict(X_test)

# 정확도 측정
lr_accuracy = accuracy_score(y_test, lr_pred)
print("로지스틱 회귀 모델의 정확도:", lr_accuracy)

# 분류 보고서 생성
lr_report = classification_report(y_test, lr_pred)
print(lr_report)

로지스틱 회귀 모델의 정확도: 0.38764044943820225
              precision    recall  f1-score   support

           1       0.83      0.30      0.44       130
           2       0.41      0.56      0.47        36
           3       0.12      0.83      0.21        12

    accuracy                           0.39       178
   macro avg       0.45      0.56      0.37       178
weighted avg       0.70      0.39      0.43       178



c:\Users\김숙경\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


이제, 생성한 모델을 사용해보겠습니다.

2번에서 행성했던 predict_survival 함수를 사용하여 예측값과 확률을 구합니다.

아래는 어떤 사람에 대한 정보입니다. 생성한 lr 모델을 이용해서 이 사람이 생존할지 생존하지 못할지, 또 그 예측이 맞을 확률은 어느 정도인지 구하세요.

[ 정보 ]
- pclass: 2
- sex: 'female'
- age: 32 
- sibsp: 1
- parch: 2 
- fare: 60
- survived: 1
- initial: 'Mrs'


In [18]:
model = lr_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

result, probability

C:\Users\김숙경\AppData\Local\Temp\ipykernel_7776\1060574916.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.


('S', np.float64(0.23511793872537834))

### 의사 결정 나무
이번에는 의사 결정 나무 모델을 만들어보겠습니다. 다른 과정은 모두 똑같고, 로지스틱 회귀 모델 대신 의사 결정 나무 모델을 사용합니다.

sklearn 라이브러리는 의사 결정 나무 모델(DecisionTreeClassifier)을 제공합니다.

[ 참고 ]
의사 결정 나무 모델 생성 함수
~~~
DecisionTreeClassifier(random_state=시드값)

- random_state: 임의의 숫자로 설정된 시드로, 어떤 숫자를 사용해도 상관없습니다.

- max_depth: 트리의 최대 깊이를 제한합니다. 이 값을 작게 설정할수록 모델이 단순해집니다.

- min_samples_split: 노드를 분할하기 위해 필요한 최소한의 데이터(샘플) 개수를 지정합니다. 이 값을 높게 설정하면 트리의 성장을 억제하여 과적합을 방지하는 효과가 있습니다.

- min_samples_leaf: 분할 후, 리프 노드가 가져야 하는 최소한의 데이터(샘플) 개수를 지정합니다. min_samples_split과 비슷하지만, 분할 이후의 조건을 검사합니다. 예를 들어, 어떤 노드를 분할했을 때 자식 노드 중 하나의 데이터 개수가 이 값보다 작아진다면 해당 분할은 수행되지 않습니다. 이 역시 모델을 부드럽게(smoothing) 하고 과적합을 막는 역할을 합니다.

- ccp_alpha:(Cost-Complexity Pruning) 비용 복잡도 가지치기(Pruning)에 사용되는 파라미터입니다. 값이 클수록 더 많은 가지가 잘려나가 트리가 단순해집니다.

- criterion: 노드를 분할할 때 어떤 기준으로 불순도(Impurity)를 측정할지 결정합니다. 불순도는 한 노드에 여러 클래스의 데이터가 얼마나 섞여 있는지를 나타내는 지표입니다. 'gini' 아니면 'entropy'를 선택할 수 있습니다.

- max_features: 최적의 분할을 찾기 위해 고려할 피처(변수)의 최대 개수를 지정합니다. 매 분할마다 모든 피처를 고려하는 대신, 무작위로 선택된 일부 피처 중에서만 최적의 분할 기준을 찾습니다. 이는 트리가 특정 피처에 과도하게 의존하는 것을 막아주며, 특히 피처가 매우 많을 때 과적합 방지 및 훈련 속도 향상에 도움이 됩니다.
~~~

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [19]:
# sklearn.tree에서 DecisionTreeClassifier를 import 해오세요.
from sklearn.tree import DecisionTreeClassifier

# tree_model이라는 변수에 의사 결정 나무 모델을 생성해주세요. 단, random_state는 42로 합니다.
tree_model = DecisionTreeClassifier(random_state=42)

이제 모델을 학습, 평가하고 사용해보겠습니다.

In [20]:
# 모델 학습
tree_model.fit(X_train_scaled, y_train)

# 예측 결과 생성
tree_pred = tree_model.predict(X_test_scaled)

# 정확도 측정
tree_accuracy = accuracy_score(y_test, tree_pred)
print("Decision Tree 모델의 정확도:", tree_accuracy)

# 분류 보고서 생성
tree_report = classification_report(y_test, tree_pred)
print(tree_report)

model = tree_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

result, probability

Decision Tree 모델의 정확도: 0.702247191011236
              precision    recall  f1-score   support

           1       0.77      0.88      0.82       130
           2       0.38      0.22      0.28        36
           3       0.38      0.25      0.30        12

    accuracy                           0.70       178
   macro avg       0.51      0.45      0.47       178
weighted avg       0.66      0.70      0.67       178



C:\Users\김숙경\AppData\Local\Temp\ipykernel_7776\1060574916.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.


('C', np.float64(0.0))

### 서포트 벡터 머신 (SVM)
서포트 벡터 머신 역시 같은 방식으로 모델을 만듭니다.
sklearn 라이브러리는 서포트 벡터 머신 분류기(SVC=Support Vector Classification)을 제공합니다.

[ 참고 ]
서포트 벡터 머신 모델 생성 함수
~~~
SVC(random_state=시드값, probability=True/False)

- random_state: 임의의 숫자로 설정된 시드로, 어떤 숫자를 사용해도 상관없습니다.

- probability: 모델이 클래스의 확률을 제공할 수 있도록 하는 옵션입니다.

- C: 마진의 너비와 오류 데이터(Margin Violation)를 얼마나 허용할지를 결정하는 파라미터입니다.

- kernel: 데이터를 어떤 공간에서 바라볼지 결정합니다.

- gamma: 비선형 커널에서만 의미가 있는 파라미터입니다. 하나의 데이터 샘플이 경계선에 영향을 미치는 범위를 결정합니다.
~~~

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [21]:
# sklearn.svm에서 SVC를 import 해오세요.
from sklearn.svm import SVC

# svm_model 단, random_state는 42, probability=True로 합니다.
svm_model = SVC(random_state=42,probability=True)

이제 모델을 학습, 평가하고 사용해보겠습니다.

In [22]:
# 모델 훈련
svm_model.fit(X_train_scaled, y_train)

# SVM 예측 결과 생성
svm_pred = svm_model.predict(X_test_scaled)

# 정확도 측정
svm_accuracy = accuracy_score(y_test, svm_pred)
print("SVM 모델의 정확도:", svm_accuracy)

# 분류 보고서 생성
svm_report = classification_report(y_test, svm_pred)
print(svm_report)

# 생성한 모델로 예측값과 확률 도출
model = svm_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

result, probability

SVM 모델의 정확도: 0.7415730337078652
              precision    recall  f1-score   support

           1       0.74      0.99      0.85       130
           2       0.00      0.00      0.00        36
           3       0.75      0.25      0.38        12

    accuracy                           0.74       178
   macro avg       0.50      0.41      0.41       178
weighted avg       0.59      0.74      0.65       178



c:\Users\김숙경\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\김숙경\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\김숙경\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

('S', np.float64(0.16607683639058685))

### kNN
kNN 역시 같은 방식으로 모델을 만듭니다.
sklearn 라이브러리는 kNN 모델(KNeighborsClassifier)을 제공합니다.

[ 참고 ]
kNN 모델 생성 함수
~~~
KNeighborsClassifier(n_neighbors=이웃 수)

- n_neighbors: 고려할 최근접 이웃의 개수입니다.

- weights: k개의 이웃을 찾았을 때, 그들의 의견을 어떤 가중치로 반영할지 결정합니다.

- metric: 데이터 포인트 사이의 '거리'를 어떤 방식으로 측정할지 결정합니다.
~~~

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [23]:
# sklearn.neighbors에서 KNeighborsClassifier를 import 해오세요.

from sklearn.neighbors import KNeighborsClassifier
# knn_model이라는 변수에 kNN 모델을 생성해주세요. 단, n_neighbors=5로 합니다.
knn_model =  KNeighborsClassifier(n_neighbors=5)

이제 모델을 학습, 평가하고 사용해보겠습니다.

In [24]:
# 모델 학습
knn_model.fit(X_train_scaled, y_train)

# 예측 수행
knn_pred = knn_model.predict(X_test_scaled)

# 정확도 측정
knn_accuracy = accuracy_score(y_test, knn_pred)
print("kNN 모델의 정확도 : ", knn_accuracy)

# 분류 보고서 생성
knn_report = classification_report(y_test, knn_pred)
print(knn_report)

# 생성한 모델로 예측값과 확률 도출
model = knn_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)


result, probability

kNN 모델의 정확도 :  0.7078651685393258
              precision    recall  f1-score   support

           1       0.77      0.88      0.82       130
           2       0.38      0.25      0.30        36
           3       0.50      0.25      0.33        12

    accuracy                           0.71       178
   macro avg       0.55      0.46      0.48       178
weighted avg       0.67      0.71      0.68       178



C:\Users\김숙경\AppData\Local\Temp\ipykernel_7776\1060574916.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.


('S', np.float64(0.2))

## 하이퍼파라미터 튜닝

마지막으로 하이퍼파라미터 튜닝을 통해 모델의 성능을 개선하는 방법을 알아봅시다.

의사결정나무에 Grid Search와 Random Search를 적용해봅시다!

[ 참고 ]
의사결정나무 모델 주요 하이퍼파라미터
~~~
DecisionTreeClassifier()

- max_depth: 트리의 최대 깊이를 제한합니다. 이 값을 작게 설정할수록 모델이 단순해집니다.

- min_samples_split: 노드를 분할하기 위해 필요한 최소한의 데이터(샘플) 개수를 지정합니다. 이 값을 높게 설정하면 트리의 성장을 억제하여 과적합을 방지하는 효과가 있습니다.

- min_samples_leaf: 분할 후, 리프 노드가 가져야 하는 최소한의 데이터(샘플) 개수를 지정합니다. min_samples_split과 비슷하지만, 분할 이후의 조건을 검사합니다. 예를 들어, 어떤 노드를 분할했을 때 자식 노드 중 하나의 데이터 개수가 이 값보다 작아진다면 해당 분할은 수행되지 않습니다. 이 역시 모델을 부드럽게(smoothing) 하고 과적합을 막는 역할을 합니다.

- ccp_alpha:(Cost-Complexity Pruning) 비용 복잡도 가지치기(Pruning)에 사용되는 파라미터입니다. 값이 클수록 더 많은 가지가 잘려나가 트리가 단순해집니다.

- criterion: 노드를 분할할 때 어떤 기준으로 불순도(Impurity)를 측정할지 결정합니다. 불순도는 한 노드에 여러 클래스의 데이터가 얼마나 섞여 있는지를 나타내는 지표입니다. 'gini' 아니면 'entropy'를 선택할 수 있습니다.

- max_features: 최적의 분할을 찾기 위해 고려할 피처(변수)의 최대 개수를 지정합니다. 매 분할마다 모든 피처를 고려하는 대신, 무작위로 선택된 일부 피처 중에서만 최적의 분할 기준을 찾습니다. 이는 트리가 특정 피처에 과도하게 의존하는 것을 막아주며, 특히 피처가 매우 많을 때 과적합 방지 및 훈련 속도 향상에 도움이 됩니다.
~~~


💡 하이퍼파라미터 최적화 방법
- Grid Search: 정해진 범위에서 Hyperparameter를 모두 순회
- Random Search: 정해진 범위에서 Hyperparameter를 무작위로 탐색
- Bayesian Optimization: 사전 정보를 바탕으로 Hyperparameter 값을 확률적으로 추정하며 탐색

### Grid Search
정해진 범위에서 Hyperparameter를 모두 순회하며 가장 좋은 성능을 내는 값을 찾는 기법
- **장점**: 범위가 넓고 step이 작을수록 꼼꼼하게 전 범위를 탐색하니 최적해를 **정확히 찾을 수 있다**.
- **단점**: 시간이 너무 오래 걸린다.
- **적용**: 넓은 범위, 큰 step을 활용해 범위를 좁힌다.

[ 참고 ]
Grid Search 주요 하이퍼파라미터
~~~
GridSearchCV()

- estimator: 튜닝할 모델 객체를 지정합니다.

- param_grid: 테스트할 하이퍼파라미터들의 목록을 사전(dictionary) 형태로 전달합니다.

- cv: 교차 검증(Cross-Validation)을 어떻게 수행할지 지정합니다.

- scoring: 최적의 하이퍼파라미터를 선택하기 위한 평가 지표를 지정합니다.

- n_jobs: 튜닝을 수행할 때 사용할 CPU 코어의 개수를 지정합니다.

- verbose: 튜닝 과정에서 출력되는 메시지의 양을 조절합니다.

In [25]:
import sys
sys.getdefaultencoding()

'utf-8'

In [30]:
from sklearn.model_selection import GridSearchCV

# Decision Tree 모델 생성
tree_model = DecisionTreeClassifier(random_state=42)

# 튜닝할 하이퍼파라미터의 후보 값들 설정
param_grid = {
    'max_depth': [3, 5, 7, 10],         # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],    # 노드를 나누기 위한 최소 샘플 수
    'min_samples_leaf': [1, 3, 5]       # 리프 노드가 되기 위한 최소 샘플 수
}

# GridSearchCV 객체 생성 (cv: 5겹 교차검증)
# n_jobs=-1 은 사용 가능한 모든 CPU 코어를 사용하여 학습 속도를 높입니다.
grid_search = GridSearchCV(estimator=tree_model, param_grid=param_grid, 
                           cv=5, verbose=1, n_jobs=-1)

# 최적의 하이퍼파라미터를 찾기 위해 모델 학습
grid_search.fit( X_train_scaled, y_train)

# 최적의 하이퍼파라미터와 그때의 최고 점수 출력
print("최적 하이퍼파라미터:", grid_search.best_params_)
print(f"최고 교차검증 정확도: {grid_search.best_score_:.4f}")

# Grid Search가 찾은 최적의 모델을 저장
best_tree_model = grid_search.best_estimator_

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

In [ ]:
# 최적 모델로 예측 결과 생성
tree_pred = best_tree_model.predict(X_test_scaled)

# 정확도 측정
tree_accuracy = accuracy_score(y_test, tree_pred)
print("\n튜닝된 Decision Tree 모델의 정확도:", tree_accuracy)

# 분류 보고서 생성
tree_report = classification_report(y_test, tree_pred)
print("\n[튜닝된 모델의 분류 보고서]")
print(tree_report)


튜닝된 Decision Tree 모델의 정확도: 0.7696629213483146

[튜닝된 모델의 분류 보고서]
              precision    recall  f1-score   support

           1       0.80      0.93      0.86       130
           2       0.63      0.33      0.44        36
           3       0.57      0.33      0.42        12

    accuracy                           0.77       178
   macro avg       0.67      0.53      0.57       178
weighted avg       0.75      0.77      0.74       178



In [ ]:
# 최적화된 모델인 'best_tree_model'을 사용합니다.
model = best_tree_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

print("\n[새로운 데이터에 대한 예측 결과]")
print("예측 결과:", result)
print("생존 확률:", probability)


[새로운 데이터에 대한 예측 결과]
예측 결과: S
생존 확률: 0.20469798657718122


/var/folders/sv/vq32hm_j3vx268sn_knwm_hm0000gn/T/ipykernel_10568/1060574916.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.


### Random Search
정해진 범위에서 Hyperparameter를 **무작위**로 탐색해 가장 좋은 성능을 내는 값을 찾는 기법
- **장점**: 속도가 Grid Search보다 빠르다.
- **단점**: 무작위라는 한계 때문에 **정확도가 떨어진다**. 따라서 Grid Search나 Bayesian Optimization에 비해 사용 빈도가 적다.

[ 참고 ]
Grid Search 주요 하이퍼파라미터
~~~
RandomizedSearchCV()

- estimator: 튜닝할 모델 객체를 지정합니다.

- param_distributions: 탐색할 하이퍼파라미터의 분포 또는 목록을 사전(dictionary) 형태로 지정합니다.

- n_iter: 지정된 param_distributions에서 몇 개의 하이퍼파라미터 조합을 무작위로 추출하여 테스트할지 그 횟수를 결정합니다.

- random_state: 결과의 재현성(reproducibility)을 위한 파라미터입니다.

- cv: 교차 검증 분할 개수

- scoring: 최적 모델 선택을 위한 평가 지표

- n_jobs: 사용할 CPU 코어 수

- verbose: 진행 과정 출력 메시지 양

In [29]:
from sklearn.model_selection import RandomizedSearchCV

# 기본 Decision Tree 모델 생성
tree_model = DecisionTreeClassifier(random_state=42)

# 튜닝할 하이퍼파라미터의 후보 값들 설정
param_dist = {
    'max_depth': [3, 5, 7, 10, 15, 20, None], # None은 깊이 제한 없음을 의미
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'criterion': ['gini', 'entropy']
}

# RandomizedSearchCV 객체 생성
# n_iter: 시도할 파라미터 조합의 수 (많을수록 좋은 조합을 찾을 확률이 높지만, 시간이 오래 걸림)
# cv: 5겹 교차검증
# n_jobs=-1: 사용 가능한 모든 CPU 코어를 사용하여 학습 속도 향상
random_search = RandomizedSearchCV(estimator=tree_model, 
                                   param_distributions=param_dist,
                                   n_iter=100, # 100개의 파라미터 조합을 무작위로 테스트합니다.
                                   cv=5, 
                                   verbose=1, 
                                   random_state=42, # 결과를 재현하기 위해 random_state 설정
                                   n_jobs=-1)

# 최적의 하이퍼파라미터를 찾기 위해 모델 학습
random_search.fit(X_train_scaled, y_train)

# 최적의 하이퍼파라미터와 그때의 최고 점수 출력
print("최적 하이퍼파라미터:", random_search.best_params_)
print(f"최고 교차검증 정확도: {random_search.best_score_:.4f}")

# Random Search가 찾은 최적의 모델을 저장
best_tree_model = random_search.best_estimator_

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

In [ ]:
# 최적 모델로 예측 결과 생성
tree_pred = best_tree_model.predict(X_test_scaled)

# 정확도 측정
tree_accuracy = accuracy_score(y_test, tree_pred)
print("\n튜닝된 Decision Tree 모델의 정확도:", tree_accuracy)

# 분류 보고서 생성
tree_report = classification_report(y_test, tree_pred)
print("\n[튜닝된 모델의 분류 보고서]")
print(tree_report)


튜닝된 Decision Tree 모델의 정확도: 0.7696629213483146

[튜닝된 모델의 분류 보고서]
              precision    recall  f1-score   support

           1       0.80      0.93      0.86       130
           2       0.63      0.33      0.44        36
           3       0.57      0.33      0.42        12

    accuracy                           0.77       178
   macro avg       0.67      0.53      0.57       178
weighted avg       0.75      0.77      0.74       178



In [ ]:
# 3. 튜닝된 최적 모델로 새로운 데이터 예측
model = best_tree_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

print("\n[새로운 데이터에 대한 예측 결과]")
print("예측 결과:", result)
print("생존 확률:", probability)


[새로운 데이터에 대한 예측 결과]
예측 결과: S
생존 확률: 0.20469798657718122


/var/folders/sv/vq32hm_j3vx268sn_knwm_hm0000gn/T/ipykernel_10568/1060574916.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.


## 끝으로...

이번 세션에서 배운 모델들(로지스틱 회귀, 의사결정나무, SVM, kNN)의 하이퍼파라미터를 튜닝하여 가장 성능이 좋은 모델을 만들어주세요!

정확도가 가장 높은 1등에게는 소정의 상품이 지급될 예정입니다!